In [1]:
import pandas as pd
import polars as pl
import geopandas as gpd
from geodatasets import get_path
from geopandas import GeoSeries
import json
import pyproj
from shapely.geometry import Point

In [2]:
# nyc = gpd.read_file(get_path('nybb'))

In [3]:
# nyc.columns = ['code', 'borough', 'Shape_Leng', 'Shape_Area', 'geometry']

In [4]:
# nyc.to_parquet("data/nyc.parquet")

In [5]:
nyc = gpd.read_parquet("data/nyc.parquet")
nyc

,code,borough,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330470.010332,1.623820e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227...."
1,4,Queens,896344.047763,3.045213e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957..."
2,3,Brooklyn,741080.523166,1.937479e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."
3,1,Manhattan,359299.096471,6.364715e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940...."
4,2,Bronx,464392.991824,1.186925e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278..."


In [6]:
df = pd.read_parquet("data/borough.parquet")
df

,borough,number_of_persons_injured,number_of_persons_killed,number_of_casualty,number_of_crash
0,BRONX,72466,291,72757,216786
1,BROOKLYN,157539,648,158187,466928
2,MANHATTAN,70563,351,70914,326658
3,QUEENS,120793,545,121338,392867
4,STATEN ISLAND,17809,97,17906,61409


In [7]:
df['code'] = [2,3,1, 4, 5]
df.head()

,borough,number_of_persons_injured,number_of_persons_killed,number_of_casualty,number_of_crash,code
0,BRONX,72466,291,72757,216786,2
1,BROOKLYN,157539,648,158187,466928,3
2,MANHATTAN,70563,351,70914,326658,1
3,QUEENS,120793,545,121338,392867,4
4,STATEN ISLAND,17809,97,17906,61409,5


In [8]:
# merged = nyc.merge(df.drop("borough", axis=1), on='code')

In [9]:
projected = nyc.to_crs(pyproj.CRS.from_epsg(4326))
projected['center'] = nyc.centroid.to_crs(pyproj.CRS.from_epsg(4326)).apply(lambda x: (x.y, x.x))
projected = projected.rename(columns={"Shape_Leng": "length", "Shape_Area": "area"})
projected.head()

,code,borough,length,area,geometry,center
0,5,Staten Island,330470.010332,1.623820e+09,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ...","(40.58084706463499, -74.15339881842505)"
1,4,Queens,896344.047763,3.045213e+09,"MULTIPOLYGON (((-73.83668 40.59495, -73.83678 ...","(40.70757313095317, -73.8184734498325)"
2,3,Brooklyn,741080.523166,1.937479e+09,"MULTIPOLYGON (((-73.86706 40.58209, -73.86769 ...","(40.64472045442235, -73.94768088830052)"
3,1,Manhattan,359299.096471,6.364715e+08,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ...","(40.77724938433267, -73.96719027867236)"
4,2,Bronx,464392.991824,1.186925e+09,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ...","(40.85261971171482, -73.8665269081608)"


In [10]:
projected.to_parquet("data/nyc_projected.parquet")

In [11]:
# merged.to_parquet("data/heatmap.parquet")
# merged.to_csv("data/heatmap.csv")
# with open("data/heatmap.json", mode="w") as f:
#     json.dump(merged.to_json(), f)

In [12]:
# merged.plot(column="count",cmap='Oranges', legend=True)

In [13]:
# merged.to_file("data/heatmap.geojson", driver="GeoJSON") 

In [14]:
clean = pl.read_parquet("data/clean_map.parquet")
clean = clean.to_pandas()
# clean['coordinate'] = clean['coordinate'].apply(lambda x: Point(x))
clean.head()

,date,borough,latitude,longitude,month,year,time,hour,coordinate,number_of_persons_injured,number_of_persons_killed,number_of_casualty,code
0,2021-09-11,BROOKLYN,40.667202,-73.866500,9,2021,09:35:00,9,"[40.667202, -73.8665]",0,0,0,3
1,2021-12-14,BROOKLYN,40.683304,-73.917274,12,2021,08:13:00,8,"[40.683304, -73.917274]",0,0,0,3
2,2021-12-14,BRONX,40.868160,-73.831480,12,2021,08:17:00,8,"[40.86816, -73.83148]",2,0,2,2
3,2021-12-14,BROOKLYN,40.671720,-73.897100,12,2021,21:10:00,21,"[40.67172, -73.8971]",0,0,0,3
4,2021-12-14,MANHATTAN,40.751440,-73.973970,12,2021,14:58:00,14,"[40.75144, -73.97397]",0,0,0,1


In [15]:
merged = clean.drop("borough", axis=1).merge(projected.drop(["geometry", "length", "center"], axis=1), on=["code"])
merged.head()

,date,latitude,longitude,month,year,time,hour,coordinate,number_of_persons_injured,number_of_persons_killed,number_of_casualty,code,borough,area
0,2021-09-11,40.667202,-73.866500,9,2021,09:35:00,9,"[40.667202, -73.8665]",0,0,0,3,Brooklyn,1.937479e+09
1,2021-12-14,40.683304,-73.917274,12,2021,08:13:00,8,"[40.683304, -73.917274]",0,0,0,3,Brooklyn,1.937479e+09
2,2021-12-14,40.868160,-73.831480,12,2021,08:17:00,8,"[40.86816, -73.83148]",2,0,2,2,Bronx,1.186925e+09
3,2021-12-14,40.671720,-73.897100,12,2021,21:10:00,21,"[40.67172, -73.8971]",0,0,0,3,Brooklyn,1.937479e+09
4,2021-12-14,40.751440,-73.973970,12,2021,14:58:00,14,"[40.75144, -73.97397]",0,0,0,1,Manhattan,6.364715e+08


In [16]:
merged.shape

(1411055, 14)

In [17]:
merged.head()

,date,latitude,longitude,month,year,time,hour,coordinate,number_of_persons_injured,number_of_persons_killed,number_of_casualty,code,borough,area
0,2021-09-11,40.667202,-73.866500,9,2021,09:35:00,9,"[40.667202, -73.8665]",0,0,0,3,Brooklyn,1.937479e+09
1,2021-12-14,40.683304,-73.917274,12,2021,08:13:00,8,"[40.683304, -73.917274]",0,0,0,3,Brooklyn,1.937479e+09
2,2021-12-14,40.868160,-73.831480,12,2021,08:17:00,8,"[40.86816, -73.83148]",2,0,2,2,Bronx,1.186925e+09
3,2021-12-14,40.671720,-73.897100,12,2021,21:10:00,21,"[40.67172, -73.8971]",0,0,0,3,Brooklyn,1.937479e+09
4,2021-12-14,40.751440,-73.973970,12,2021,14:58:00,14,"[40.75144, -73.97397]",0,0,0,1,Manhattan,6.364715e+08


In [18]:
# merged['inner_distance'] = merged.apply(lambda x: x['coordinate'].distance(x['center']), axis=1)
merged.head()

,date,latitude,longitude,month,year,time,hour,coordinate,number_of_persons_injured,number_of_persons_killed,number_of_casualty,code,borough,area
0,2021-09-11,40.667202,-73.866500,9,2021,09:35:00,9,"[40.667202, -73.8665]",0,0,0,3,Brooklyn,1.937479e+09
1,2021-12-14,40.683304,-73.917274,12,2021,08:13:00,8,"[40.683304, -73.917274]",0,0,0,3,Brooklyn,1.937479e+09
2,2021-12-14,40.868160,-73.831480,12,2021,08:17:00,8,"[40.86816, -73.83148]",2,0,2,2,Bronx,1.186925e+09
3,2021-12-14,40.671720,-73.897100,12,2021,21:10:00,21,"[40.67172, -73.8971]",0,0,0,3,Brooklyn,1.937479e+09
4,2021-12-14,40.751440,-73.973970,12,2021,14:58:00,14,"[40.75144, -73.97397]",0,0,0,1,Manhattan,6.364715e+08


In [19]:
merged.dtypes

date                         datetime64[ms]
latitude                            float64
longitude                           float64
month                                  int8
year                                  int32
time                                 object
hour                                   int8
coordinate                           object
number_of_persons_injured             int64
number_of_persons_killed              int64
number_of_casualty                    int64
code                                   int8
borough                              object
area                                float64
dtype: object

In [20]:
# merged['inner_distance'] = merged['inner_distance']/1000
# merged['coordinate'] = merged['coordinate'].apply(lambda x: (x.x, x.y))
# merged['center'] = merged['center'].apply(lambda x: (x.y, x.x))
merged.head()

,date,latitude,longitude,month,year,time,hour,coordinate,number_of_persons_injured,number_of_persons_killed,number_of_casualty,code,borough,area
0,2021-09-11,40.667202,-73.866500,9,2021,09:35:00,9,"[40.667202, -73.8665]",0,0,0,3,Brooklyn,1.937479e+09
1,2021-12-14,40.683304,-73.917274,12,2021,08:13:00,8,"[40.683304, -73.917274]",0,0,0,3,Brooklyn,1.937479e+09
2,2021-12-14,40.868160,-73.831480,12,2021,08:17:00,8,"[40.86816, -73.83148]",2,0,2,2,Bronx,1.186925e+09
3,2021-12-14,40.671720,-73.897100,12,2021,21:10:00,21,"[40.67172, -73.8971]",0,0,0,3,Brooklyn,1.937479e+09
4,2021-12-14,40.751440,-73.973970,12,2021,14:58:00,14,"[40.75144, -73.97397]",0,0,0,1,Manhattan,6.364715e+08


In [21]:
# merged = merged.drop(["distance"], axis=1)

In [22]:
merged.to_parquet("data/clean_map.parquet")

In [23]:
merged.head()

,date,latitude,longitude,month,year,time,hour,coordinate,number_of_persons_injured,number_of_persons_killed,number_of_casualty,code,borough,area
0,2021-09-11,40.667202,-73.866500,9,2021,09:35:00,9,"[40.667202, -73.8665]",0,0,0,3,Brooklyn,1.937479e+09
1,2021-12-14,40.683304,-73.917274,12,2021,08:13:00,8,"[40.683304, -73.917274]",0,0,0,3,Brooklyn,1.937479e+09
2,2021-12-14,40.868160,-73.831480,12,2021,08:17:00,8,"[40.86816, -73.83148]",2,0,2,2,Bronx,1.186925e+09
3,2021-12-14,40.671720,-73.897100,12,2021,21:10:00,21,"[40.67172, -73.8971]",0,0,0,3,Brooklyn,1.937479e+09
4,2021-12-14,40.751440,-73.973970,12,2021,14:58:00,14,"[40.75144, -73.97397]",0,0,0,1,Manhattan,6.364715e+08


In [24]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411055 entries, 0 to 1411054
Data columns (total 14 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   date                       1411055 non-null  datetime64[ms]
 1   latitude                   1411055 non-null  float64       
 2   longitude                  1411055 non-null  float64       
 3   month                      1411055 non-null  int8          
 4   year                       1411055 non-null  int32         
 5   time                       1411055 non-null  object        
 6   hour                       1411055 non-null  int8          
 7   coordinate                 1411055 non-null  object        
 8   number_of_persons_injured  1411055 non-null  int64         
 9   number_of_persons_killed   1411055 non-null  int64         
 10  number_of_casualty         1411055 non-null  int64         
 11  code                       1411055 no